In [12]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Download NLTK resources (if not already downloaded)
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [41]:
import pandas as pd
desired_companies = [
 'gap',
 'banana-republic',
 'old-navy',
 'h-and-m',
 'zara',
 'macys',
 'dillards',
 'urban-outfitters',
 'american-eagle-outfitters',
 'express-clothing',
 'abercrombie-and-fitch']

In [42]:
key = "c0945b0821364c2299fd5296bdfd961e" #"<paste-your-text-analytics-key-here>"
endpoint = "https://review-sentiments.cognitiveservices.azure.com/" #"<paste-your-text-analytics-endpoint-here>"

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

In [104]:
client = authenticate_client()

def sentiment_analysis(client, reviews):

    isFailure = False
    documents = reviews
    response = None
    try:
        if len(documents) <= 5120:
            response = client.analyze_sentiment(documents=documents)[0]
        else:
            isFailure =  True
    except:
        isFailure =  True
    print(response)
    
    sentiments = []
    
    if not isFailure:
        for idx, sentence in enumerate(response.sentences):
            # sentiments.append(sentence['sentiment'])
            sentiments.append(sentence['confidence_scores'])
    

    if not isFailure: 
        return sentiments[0].negative
    else:
        return 0 

In [89]:
review = [None]
a = sentiment_analysis(client, review)

None


In [90]:
a

0

In [111]:
desired_companies = [
 'gap'
 ]

In [112]:
# Load the Excel file
excel_file = pd.ExcelFile("reviews_mourya.xlsx")

# Get the list of sheet names in the Excel file
sheet_names = excel_file.sheet_names

# Initialize a list to store DataFrames for each sheet
dfs_with_sentiment = []

# Loop through each sheet and perform pandas analysis
for sheet_name in desired_companies:
    # Read the sheet into a DataFrame
    df_old = excel_file.parse(sheet_name)
    df_old["Date"] = pd.to_datetime(df_old["Date"], format="%b %d, %Y")
    df = df_old[df_old["Date"] < "Jan 1, 2020"]
    
    # # Apply the sentiment analysis function to the "Review" column
    df["Sentiment"] = df.apply(lambda row: sentiment_analysis(client, [row["Review"]]), axis=1)

    df["SheetName"] = sheet_name
    dfs_with_sentiment.append(df)

combined_df = pd.concat(dfs_with_sentiment, ignore_index=True)
    
    # # Append the DataFrame to the list
    # dfs_with_sentiment.append(df)
    # break

{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.02, neutral=0.07, negative=0.91), 'sentences': [SentenceSentiment(text=                 I paid extra to have an order I paid for arrive early, however did not receive my order. , sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.02, negative=0.98), length=106, offset=0, mined_opinions=[]), SentenceSentiment(text=Gap did nothing except keep my money while I'm trying to get my money back since I received no products. , sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.03, negative=0.96), length=105, offset=106, mined_opinions=[]), SentenceSentiment(text=They didn't even bother to call the shipping company to verify: they literally did nothing but keep my money! , sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.08, neutral=0.22, negative=0.7), length=110, off

C:\Users\Owner\AppData\Local\Temp\ipykernel_5584\3836470423.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sentiment"] = df.apply(lambda row: sentiment_analysis(client, [row["Review"]]), axis=1)
C:\Users\Owner\AppData\Local\Temp\ipykernel_5584\3836470423.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SheetName"] = sheet_name


In [100]:
zara = dfs_with_sentiment[0]

In [110]:
old_navy = dfs_with_sentiment[0]

In [113]:
gap = dfs_with_sentiment[0]

In [115]:
gap.to_csv("gap_sentiment.csv")
zara.to_csv("zara_sentiment.csv")
old_navy.to_csv("old_navy_sentiment.csv")

In [49]:
dfs_with_sentiment[0]

,Author,Subject,Date,Rating,Review
4,PissedConsumer4937604,My order didn’t take my promo code,"Aug 11, 2023",1.2,\n Called customer service and ...
5,Rayland Ezd,Make a payment over the phone,"Aug 24, 2023",NaN,\n I have made several attempts...
6,Merdis Bxs,Fraude,"Aug 02, 2023",NaN,\n Me cobraron una orden de mas...
16,Teresa J Jso,Product,"Dec 07, 2022",NaN,\n I purchased a leather vest o...
19,fernanda f Mpa,NaN,"Feb 25, 2023",1.0,\n I cancelled ...
...,...,...,...,...,...
517,PissedConsumer569621,Shipping takes too long!!,"Dec 13, 2014",NaN,\n I placed an order with Baby ...
518,PissedConsumer566852,NaN,"Dec 07, 2014",NaN,\n Sunday I wen...
521,Brailee Mfm,NaN,"Feb 22, 2013",NaN,NaN
522,Julie P Fbb,Deceptive Practices at Gap Credit Services,"Dec 08, 2012",NaN,\n Gap credit services has been...


In [116]:
gap

,Author,Subject,Date,Rating,Review,Sentiment,SheetName
503,Karen Ios,DO NOT PURCHASE FROM GAP,2019-11-19,1.0,\n I paid extra to have an orde...,0.98,gap
505,Yashika K Nfd,Dissatisfaction with gap sizes,2019-12-28,NaN,\n The sizes are always unpredi...,0.25,gap
506,Trudie Fxu,"Cheap fabric, expensive prices",2019-10-23,3.9,NaN,0.00,gap
507,Adriana H Qjw,First Experience/Will be My Last!,2019-05-18,2.3,\n Ordered 6 tee shirts online....,0.02,gap
508,Karan Qnt,Worst ever experience with GAP,2018-12-14,1.0,\n Worst shopping experience wi...,0.99,gap
509,Mike H Hab,Glitch on line,2018-12-09,1.4,\n Paid for a purchase on my ga...,0.05,gap
510,PissedConsumer1271745,Terrible service,2018-06-04,1.4,\n Got a gap credit card order ...,0.39,gap
511,PissedConsumer1281710,Mean and rude associates,2018-06-16,NaN,\n I went to the store to retur...,0.99,gap
512,PissedConsumer1231615,Worker using the *** word,2018-04-11,3.2,"\n Yeaterday at GAP in Destin, ...",0.26,gap
513,PissedConsumer971146,"Short Review on December 11, 2016",2016-12-11,3.0,\n Order on line just received ...,0.02,gap


In [123]:

def get_average_sentiment_by_year(df):

    # Convert the "Date" column to datetime
    df["Date"] = pd.to_datetime(df["Date"])

    # Extract the year from the "Date" column
    df["Year"] = df["Date"].dt.year

    # Group by year and calculate the average sentiment score for each year
    average_sentiment_by_year = df.groupby("Year")["Sentiment"].mean()

    # Reset the index to have "Year" as a column
    average_sentiment_by_year = average_sentiment_by_year.reset_index()

    return average_sentiment_by_year

In [124]:
negative_sentiment_gap = get_average_sentiment_by_year(gap)
negative_sentiment_zara = get_average_sentiment_by_year(zara)


C:\Users\Owner\AppData\Local\Temp\ipykernel_5584\2353348624.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = pd.to_datetime(df["Date"])
C:\Users\Owner\AppData\Local\Temp\ipykernel_5584\2353348624.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"] = df["Date"].dt.year


In [125]:
negative_sentiment_gap

,Year,Sentiment
0,2008,0.2600
1,2010,0.4700
2,2012,0.8000
3,2013,0.0000
4,2014,0.2175
5,2016,0.0500
6,2018,0.5360
7,2019,0.3125


In [126]:
negative_sentiment_zara

,Year,Sentiment
0,2012,0.990000
1,2013,0.495000
2,2014,0.570000
3,2015,0.526207
4,2016,0.569565
5,2017,0.554792
6,2018,0.507778
7,2019,0.421864
